In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

from sklearn import metrics
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima.model_selection import train_test_split as time_train_test_split
from xgboost import XGBRegressor
from xgboost import plot_importance
from datetime import timedelta

import xgboost as xgb
from sklearn.metrics import mean_squared_error
color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

ModuleNotFoundError: No module named 'pmdarima'

In [ ]:
df_month = pd.read_csv(
    './data/Road Crash Statistics in Quezon City (2018-2022) - Classification by Month per case basis .csv' , index_col=False)
df_hour = pd.read_csv(
    './data/Road Crash Statistics in Quezon City (2018-2022) - Classification by Time of Day.csv', index_col=False )

In [ ]:
df_month.head()

In [ ]:
df_hour.head()

## Title: Data Cleaning

In [ ]:
DEPENDENT_VARIABLE = "Grand Total"

In [ ]:
df_month[DEPENDENT_VARIABLE] =df_month[DEPENDENT_VARIABLE].fillna(0)
df_month[DEPENDENT_VARIABLE] = df_month[DEPENDENT_VARIABLE].apply(
    lambda x: 0 if x == 0 or x == '' or x == ' ' 
    else float ( 
        str(x).replace(',','').replace('',''))
    )

In [ ]:
df_hour[DEPENDENT_VARIABLE] =df_hour[DEPENDENT_VARIABLE].fillna(0)
df_hour[DEPENDENT_VARIABLE] = df_hour[DEPENDENT_VARIABLE].apply(
    lambda x: 0 if x == 0 or x == '' or x == ' ' 
    else float ( 
        str(x).replace(',','').replace('',''))
    )

In [ ]:
df_month.info()

In [ ]:
df_hour.info()

In [ ]:
df_month.describe()

In [ ]:
df_hour.describe()

In [ ]:
df_month[DEPENDENT_VARIABLE].plot(style='-.',
    figsize=(15, 5),
    )
plt.show()

In [ ]:
df_hour[DEPENDENT_VARIABLE].plot(style='-.',
                        figsize=(15, 5),
                        )
plt.show()

In [ ]:
df_month[DEPENDENT_VARIABLE].plot(kind='hist', bins=100);

In [ ]:
df_hour[DEPENDENT_VARIABLE].plot(kind='hist', bins=100);

In [ ]:
# ----------- Hour -----------
df_hour_X = df_hour.copy()

# Test Size = 20%
train_hour, test_hour = time_train_test_split(df_hour_X, test_size=int(len(df_month)*0.42))

train_hour = pd.DataFrame(train_hour)
test_hour = pd.DataFrame(test_hour)

# ----------- Monthly -----------
df_month_X = df_month.copy()

# Test Size = 20%
train_month, test_month = time_train_test_split(df_month_X, test_size=int(len(df_month)*0.2))

train_month = pd.DataFrame(train_month)
test_month = pd.DataFrame(test_month)


In [ ]:
test_hour.head(24)

In [ ]:
test_month.head(12)

In [ ]:
overview_Train_Test_Data = test_hour[[DEPENDENT_VARIABLE]].rename(columns={DEPENDENT_VARIABLE: 'TEST SET'}) .join( train_hour[[DEPENDENT_VARIABLE]].rename(columns={DEPENDENT_VARIABLE: 'TRAINING SET'}), how='outer').plot(figsize=(15,5), title='Overview Train Test Data', style='-.')

In [ ]:
train_month.head()

In [ ]:
train_hour.head()

In [ ]:
dtime_day = {
    "00:00-00:59": "Night",
    "01:00-01:59": "Night",
    "02:00-02:59": "Night",
    "03:00-03:59": "Night",
    "04:00-04:59": "Night",
    "05:00-05:59": "Morning",
    "06:00-06:59": "Morning",
    "07:00-07:59": "Morning",
    "08:00-08:59": "Morning",
    "09:00-09:59": "Morning",
    "10:00-10:59": "Morning",
    "11:00-11:59": "Morning",
    "12:00-12:59": "Afternoon",
    "13:00-13:59": "Afternoon",
    "14:00-14:59": "Afternoon",
    "15:00-15:59": "Afternoon",
    "16:00-16:59": "Afternoon",
    "17:00-17:59": "Afternoon",
    "18:00-18:59": "Evening",
    "19:00-19:59": "Evening",
    "20:00-20:59": "Night",
    "21:00-21:59": "Night",
    "22:00-22:59": "Night",
    "23:00-23:59": "Night",
    "Time Not Stated": "unkown",
}

# 0 - Not rush hour 
# 1 - Rush hour
'''
Worst traffic is at 7AM to 9AM and on 5PM to 9PM
Medium traffic is at 9AM to 10AM and on 9PM to 11PM
Low Traffic rest of the Hour
'''
rush_hour = {
    "00:00-00:59":  "Low",
    "01:00-01:59":  "Low",
    "02:00-02:59": "Low",
    "03:00-03:59": "Low",
    "04:00-04:59": "Low",
    "05:00-05:59": "Low",
    "06:00-06:59": "Low",
    "07:00-07:59": "High",
    "08:00-08:59": "High",
    "09:00-09:59": "Medium",
    "10:00-10:59": "Low",
    "11:00-11:59": "Low",
    "12:00-12:59": "Low",
    "13:00-13:59": "Low",
    "14:00-14:59": "Low",
    "15:00-15:59": "Low",
    "16:00-16:59": "Low",
    "17:00-17:59": "High",
    "18:00-18:59": "High",
    "19:00-19:59": "High",
    "20:00-20:59": "High",
    "21:00-21:59": "Medium",
    "22:00-22:59": "Medium",
    "23:00-23:59": "Medium",
    "Time Not Stated": "Neutral",
}
from dateutil.relativedelta import relativedelta, MO
def create_features_monthly(df, target_variable):
    df['Year'] = df['Year'].apply(lambda x: int(x))
    X = df[[
        'Year', 'Month'
    ]]
    X = pd.get_dummies(X, columns = ['Month'])
    if target_variable:
        y = df[target_variable]
        return X, y
    return X


def create_features_hourly(df, target_variable):
    def hourmin_decimal(time, index):
        if time == 'Time Not Stated': 
            return -1 
        time = time.split('-')[index].split(':')
        hour = int(time[0])
        minutes = int(time[-1])
        return int(hour) * 3600 + int(minutes) * 60

    df['Year'] = df['Year'].apply(lambda x: int(x))
    df['Day Time'] = df['Time Hour'].apply(lambda x:  dtime_day[x])
    df['Hour Start'] = df['Time Hour'].apply(lambda x: hourmin_decimal(x,0))
    df['Hour End'] = df['Time Hour'].apply(lambda x: hourmin_decimal(x,-1))
    df['Traffic'] = df['Time Hour'].apply(lambda x: rush_hour[x])
    X = df[[
        'Year', 'Day Time', 'Hour Start', 'Hour End', 'Traffic'
    ]]
    X = pd.get_dummies(X, columns = ['Day Time', 'Traffic'])
    if target_variable:
        y = df[target_variable]
        return X, y
    return X



def mean_absolute_percentage_error_func(y_true, y_pred):
    '''
    Calculate the mean absolute percentage error as a metric for evaluation
    
    Args:
        y_true (float64): Y values for the dependent variable (test part), numpy array of floats 
        y_pred (float64): Predicted values for the dependen variable (test parrt), numpy array of floats
    
    Returns:
        Mean absolute percentage error 
    '''    
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true))
    return mape


def timeseries_evaluation_metrics_func(y_true, y_pred):
    '''
    Calculate the following evaluation metrics:
        - MSE
        - MAE
        - RMSE
        - MAPE
        - R²
    
    Args:
        y_true (float64): Y values for the dependent variable (test part), numpy array of floats 
        y_pred (float64): Predicted values for the dependen variable (test parrt), numpy array of floats
    
    Returns:
        MSE, MAE, RMSE, MAPE and R² 
    '''    
    #print('Evaluation metric results: ')
    print(f'MSE is : {metrics.mean_squared_error(y_true, y_pred)}')
    print(f'MAE is : {metrics.mean_absolute_error(y_true, y_pred)}')
    print(f'RMSE is : {np.sqrt(metrics.mean_squared_error(y_true, y_pred))}')
    print(f'MAPE is : {mean_absolute_percentage_error_func(y_true, y_pred)}')
    print(f'R2 is : {metrics.r2_score(y_true, y_pred)}',end='\n\n')


In [ ]:
train_hour_copy = train_hour.copy()
test_hour_copy = test_hour.copy()

hour_trainX, hour_trainY = create_features_hourly(train_hour_copy, target_variable=DEPENDENT_VARIABLE)
hour_testX, hour_testY = create_features_hourly(test_hour_copy, target_variable=DEPENDENT_VARIABLE)

In [ ]:
train_month_copy = train_month.copy()
test_month_copy = test_month.copy()

month_trainX, month_trainY = create_features_monthly(train_month_copy, target_variable=DEPENDENT_VARIABLE)
month_testX, month_testY = create_features_monthly(test_month_copy, target_variable=DEPENDENT_VARIABLE)

In [ ]:
hour_trainX.head()

In [ ]:
hour_trainY

In [ ]:
month_trainX.head()

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb